In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_columns = 100
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from fuzzywuzzy import fuzz
import datetime

#for image generation
from scipy import stats
from scipy.special import expit
import matplotlib.image as mpimg

import time
from tqdm import tqdm_notebook

/Users/whpowell/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
## load cleaned data frame


In [3]:
## codes for data viz
# courtesy of CPMP (https://www.kaggle.com/cpmpml/initial-wrangling-voronoi-areas-in-python)
# with a few tweaks

def create_football_field(df,linenumbers=True,
                          highlight_line=False,
                          figsize=(12*2, 6.33*2)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0,  alpha=0.5)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    # yardline markers
    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    # Endzones
    ez1 = patches.Rectangle((0, 0), 10, 53.3,
                            linewidth=0.1,
                            edgecolor='r',
                            facecolor='red',
                            alpha=0.2,
                            zorder=0)
    ez2 = patches.Rectangle((110, 0), 120, 53.3,
                            linewidth=0.1,
                            edgecolor='r',
                            facecolor='blue',
                            alpha=0.2,
                            zorder=0)
    ax.add_patch(ez1)
    ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    
    # add hashes
    hash_range = range(11, 110)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = df['YardsFromOwnGoal'] + 10
        plt.plot([hl, hl], [0, 53.3], '--', color='white')
        hl = hl + df['Distance']
        plt.plot([hl, hl], [0, 53.3], '-', color='yellow')
    return fig, ax

def get_dx_dy(radian_angle, dist):
    dx = dist * np.cos(radian_angle)
    dy = dist * np.sin(radian_angle)
    return dx, dy

def show_play(play_id, plays):
    df = plays[plays.PlayId == play_id]
    Down = df.Down.values[0]
    Distance = df.Distance.values[0]
    YardLine = df.YardsFromOwnGoal.values[0]
    fig, ax = create_football_field(df, highlight_line=True)
    ax.scatter(df.loc[df['IsOnOffense']==True].X_std + 10, df.loc[df['IsOnOffense']==True].Y_std, marker='o', facecolors='none', edgecolors='blue', label='Offense', s=100)
    ax.scatter(df.loc[df['IsOnOffense']==False].X_std + 10, df.loc[df['IsOnOffense']==False].Y_std, marker='x', c='red', label='Defense', s=100)
    rusher_row = df[df.NflIdRusher == df.NflId]
    ax.scatter(rusher_row.X_std + 10, rusher_row.Y_std, color='black', s=100, label='Ball Carrier')
    yards_covered = rusher_row["Yards"].values[0]

    # add arrows to show player velocity vector
    for (x, y, Dir, S) in zip(df.X_std, df.Y_std, df.Dir, df.S):       
        dx, dy = get_dx_dy(Dir, S)
        ax.arrow(x+10, y, dx, dy, length_includes_head=False, width=0.2, color='black', alpha=0.5)
    left = 'left' if df.ToLeft.sum() > 0 else 'right'
    plt.title(f'Play #{play_id} moving to {left}, Down {Down} & {Distance} to go, LoS is {YardLine} ', fontsize=20)
    plt.legend()
    plt.show()

In [4]:
%matplotlib inline
show_play(20170910000081, cleandf)
show_play(20170910001102, cleandf)

NameError: name 'cleandf' is not defined